In [2]:
# Cell 1: scripts folder to path & import modules for whole coding
import sys
from pathlib import Path

# 1) Locate project root (one level up from this notebooks directory)
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    project_root = Path.cwd().parent
    
print(f"Project root identified as: {project_root}")

# 2) Point at your scripts folder
scripts_dir = project_root / "scripts_optimisation_times_3"
assert scripts_dir.exists(), f"Can't find scripts_optimisation at {scripts_dir}"
sys.path.insert(0, str(scripts_dir))

# 3) Import helper modules
from pv_simulate import simulate_multi_year_pv
from battery import simulate_battery_dispatch
from financial import compute_financials
from objective import evaluate_solution

# 4) Import optimiser & data libs
import numpy as np
import pandas as pd
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize

print("All modules imported successfully - can move onto next stage")

Project root identified as: /Users/petertunali/Documents/GitHub/Battery_Optimisation
All modules imported successfully - can move onto next stage


In [3]:
# ─── Cell 2: Configure paths, auto‑version outputs folder & check data ───────────
from pathlib import Path
import sys

# 1) Auto‑version your outputs folder under project_root/outputs_optimisation
base_out = project_root / "outputs_optimisation"
base_out.mkdir(exist_ok=True)

# Find existing numeric subfolders
existing = [d.name for d in base_out.iterdir() if d.is_dir() and d.name.isdigit()]
nums     = sorted(int(n) for n in existing) if existing else []
next_run = nums[-1] + 1 if nums else 1

# Create new run folder (zero‑padded)
out_dir = base_out / f"{next_run:03d}"
out_dir.mkdir()
print(f"Writing all outputs to → {out_dir}")

# 2) Locate your data directory (try several likely locations)
data_dir = project_root / "data"
if not data_dir.exists():
    for alt in [project_root / "Battery_Optimisation" / "data", Path.cwd() / "data"]:
        if alt.exists():
            data_dir = alt
            break
print(f"Using data directory → {data_dir}")

# 3) Check EPW weather files
weather_files = [
    str(data_dir / "Bonfire_2025.epw"),
    str(data_dir / "Bonfire_2040_4_5.epw"),
    str(data_dir / "Bonfire_2050_4_5.epw")
]
print("\nChecking weather files:")
for wf in weather_files:
    status = " Found" if Path(wf).exists() else "❌ Missing"
    print(f"  {status}: {wf}")

# 4) Identify demand CSV (takes first match)
demand_paths = [
    data_dir / "PV_Generation_excel.csv",
    data_dir / "Energy_Demand_and_Supply_2024.csv"
]
demand_file = None
print("\nChecking demand files:")
for p in demand_paths:
    if p.exists():
        demand_file = p
        print(f"   Found: {p}")
        break
else:
    print("  No demand file found; please check your filenames")

# 5) Quick sanity check
assert demand_file is not None, "Demand file not found – please correct path!"
assert all(Path(wf).exists() for wf in weather_files), "One or more weather files missing!"
print("\n All required data files are in place. Moving on…")


Writing all outputs to → /Users/petertunali/Documents/GitHub/Battery_Optimisation/outputs_optimisation/002
Using data directory → /Users/petertunali/Documents/GitHub/Battery_Optimisation/data

Checking weather files:
   Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2025.epw
   Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2040_4_5.epw
   Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2050_4_5.epw

Checking demand files:
   Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv

 All required data files are in place. Moving on…


In [4]:
# Cell 3: Define the PV systems and roof parameters
# Define PV system parameters
roof_params = [{
    'name': 'main_roof',
    'system_capacity_kw': 10.0,  # 10kW system
    'tilt': 10.0,
    'azimuth': 18.0,  # North-facing in Southern Hemisphere
    'shading': 43.0,  # 43% shading impact
}]

print("Roof parameters:")
for roof in roof_params:
    print(f"  - {roof['name']}: {roof['system_capacity_kw']} kW, Tilt: {roof['tilt']}°, Azimuth: {roof['azimuth']}°, Shading: {roof.get('shading', 0.0)}%")

total_pv_capacity = sum(roof['system_capacity_kw'] for roof in roof_params)
print(f"Total PV capacity: {total_pv_capacity} kW")
print("ready for next stage")

Roof parameters:
  - main_roof: 10.0 kW, Tilt: 10.0°, Azimuth: 18.0°, Shading: 43.0%
Total PV capacity: 10.0 kW
ready for next stage


In [5]:
# Cell 4: Simulate PV Gen for cell 3 roof parameters
print("Starting PV simulation...")

try:
    # Explicitly specify start years instead of extracting from filenames
    start_years = [2025, 2040, 2050]  # Match your weather files
    
    print(f"Using explicit start years: {start_years}")
    
    pv_profile = simulate_multi_year_pv(
        weather_files=weather_files,
        roof_params=roof_params,
        repeats_per_file=1,  # Simulate exactly one year per file
        start_years=start_years  # Explicitly set start years
    )
    
    # Check the results
    print(f"PV simulation completed - check years are done coorectly, as we will do 3 years * 10 for this sim")
    print(f"  - Time steps: {len(pv_profile)}")
    print(f"  - Date range: {pv_profile.index[0]} to {pv_profile.index[-1]}")
    print(f"  - Total generation: {pv_profile['simulated_kwh'].sum():.2f} kWh")
    
    # Display first few rows
    print("\nFirst few rows of PV generation:")
    print(pv_profile.head())
    print("ready for next stage")
    
except Exception as e:
    print(f" Error in PV simulation: {e}")
    raise
    print("redo, something has gone wrong - look into")
    

Starting PV simulation...
Using explicit start years: [2025, 2040, 2050]
PV simulation completed - check years are done coorectly, as we will do 3 years * 10 for this sim
  - Time steps: 52560
  - Date range: 2025-01-01 00:00:00 to 2027-12-31 23:30:00
  - Total generation: 20402.81 kWh

First few rows of PV generation:
                     simulated_kwh
2025-01-01 00:00:00            0.0
2025-01-01 00:30:00            0.0
2025-01-01 01:00:00            0.0
2025-01-01 01:30:00            0.0
2025-01-01 02:00:00            0.0
ready for next stage


In [38]:
# Cell 5: Load the demand data (was 2024, but moved to 2025 to make my life easier - demand was also copied for 3 years to mimic solar timeframe
print(f"Loading demand data from: {demand_file}")

try:
    # Load the CSV file and process as before
    demand_data = pd.read_csv(demand_file)
    time_col = demand_data.columns[0]
    demand_data[time_col] = pd.to_datetime(demand_data[time_col], dayfirst=True)
    
    # Determine demand column
    if 'Consumed Power' in demand_data.columns:
        demand_col = 'Consumed Power'
    elif len(demand_data.columns) >= 3:
        demand_col = demand_data.columns[2]
    else:
        raise ValueError("Cannot identify demand column in CSV")
    
    # Set timestamp as index
    demand_data.set_index(time_col, inplace=True)
    
    # Extract original demand profile
    original_demand = demand_data[demand_col]
    
    # Now create a new index that shifts the dates to match PV years (2025-2027)
    # while preserving the month, day, hour, minute
    years_to_create = [2025, 2026, 2027]
    shifted_demand_list = []
    
    for year in years_to_create:
        # Create a copy of the original data
        year_data = original_demand.copy()
        
        # Create a new index with the target year
        new_index = [pd.Timestamp(
            year=year,
            month=ts.month,
            day=ts.day,
            hour=ts.hour,
            minute=ts.minute
        ) for ts in original_demand.index]
        
        # Set the new index
        year_data.index = new_index
        shifted_demand_list.append(year_data)
    
    # Concatenate all years
    demand_profile = pd.concat(shifted_demand_list)
    demand_profile.sort_index(inplace=True)
    
    print(f" Demand profile created with shifted years to match PV data")
    print(f"  - Time steps: {len(demand_profile)}")
    print(f"  - Date range: {demand_profile.index[0]} to {demand_profile.index[-1]}")
    print(f"  - Total demand: {demand_profile.sum():.2f} kWh")
    
    # Display first few rows
    print("\nFirst few rows of adjusted demand data:")
    print(demand_profile.head())
    
    # Check if timestamps align with PV data
    if 'pv_profile' in globals():
        common_dates = pv_profile.index.intersection(demand_profile.index)
        print(f"\nNumber of overlapping timestamps with PV data: {len(common_dates)}")

        print("go to next stage with this timeframe dataset")
    
except Exception as e:
    print(f"Error loading demand data: {e}")
    raise

Loading demand data from: C:\Users\PeterTunali\OneDrive - CONNELLGRIFFIN PTY LIMITED\Documents\Python\Battery_Optimisation\data\PV_Generation_excel.csv
✅ Demand profile created with shifted years to match PV data
  - Time steps: 52416
  - Date range: 2025-01-01 00:00:00 to 2027-12-30 23:30:00
  - Total demand: 109212.34 kWh

First few rows of adjusted demand data:
2025-01-01 00:00:00    2.611
2025-01-01 00:30:00    2.715
2025-01-01 01:00:00    1.884
2025-01-01 01:30:00    0.000
2025-01-01 02:00:00    0.000
Name: Consumed Power, dtype: float64

Number of overlapping timestamps with PV data: 52416
go to next stage with this timeframe dataset


In [39]:
# Cell 6: Do Battery Simulatuon for above demand and cells above solar simulation - below size is just a testing start of 10.0 kW 
print("REMEMBER PETER TO CHANGE BATTERY SIZE OPTIM IN CELL 9 AND 10 TO LARGER)")


#CHECK BATTERY KW BASED OFF 

# Test battery simulation with a single size
test_battery_kwh = 10.0  # 10 kWh battery

print(f"Testing battery simulation with {test_battery_kwh} kWh battery...")

try:
    # Make sure PV and demand profiles have matching indices
    common_index = pv_profile.index.intersection(demand_profile.index)
    if len(common_index) == 0:
        print("❌ No overlapping dates between PV and demand profiles!")
        print(f"PV dates: {pv_profile.index[0]} to {pv_profile.index[-1]}")
        print(f"Demand dates: {demand_profile.index[0]} to {demand_profile.index[-1]}")
    else:
        print(f"Found {len(common_index)} overlapping time steps")
        
        # Align profiles
        aligned_pv = pv_profile.loc[common_index]
        aligned_demand = demand_profile.loc[common_index]
        
        # Run battery simulation
        if hasattr(aligned_pv, 'columns'):
            pv_series = aligned_pv['simulated_kwh']
        else:
            pv_series = aligned_pv
            
        dispatch_df, totals = simulate_battery_dispatch(
            pv_gen=pv_series,
            demand=aligned_demand,
            battery_kwh=test_battery_kwh
        )
        
        # Display results
        print(f"Battery simulation successful")
        print(f"  - Peak grid imports: {totals['total_grid_import_peak']:.2f} kWh")
        print(f"  - Off-peak grid imports: {totals['total_grid_import_offpeak']:.2f} kWh")
        print(f"  - PV exports: {totals['total_pv_export']:.2f} kWh")
        
        # Display first few rows
        print("\nFirst few rows of battery dispatch:")
        print(dispatch_df.head())
        
except Exception as e:
    print(f"Error in battery simulation: {e}")
    raise

REMEMBER PETER TO CHANGE BATTERY SIZE OPTIM IN CELL 9 AND 10 TO LARGER)
Testing battery simulation with 10.0 kWh battery...
Found 52416 overlapping time steps
Battery simulation successful
  - Peak grid imports: 28824.45 kWh
  - Off-peak grid imports: 60644.02 kWh
  - PV exports: 1110.53 kWh

First few rows of battery dispatch:
                     pv_gen  demand  pv_used  battery_charge  \
2025-01-01 00:00:00     0.0   2.611      0.0             0.0   
2025-01-01 00:30:00     0.0   2.715      0.0             0.0   
2025-01-01 01:00:00     0.0   1.884      0.0             0.0   
2025-01-01 01:30:00     0.0   0.000      0.0             0.0   
2025-01-01 02:00:00     0.0   0.000      0.0             0.0   

                     battery_discharge  battery_soc  pv_export  \
2025-01-01 00:00:00                0.0          2.0        0.0   
2025-01-01 00:30:00                0.0          2.0        0.0   
2025-01-01 01:00:00                0.0          2.0        0.0   
2025-01-01 01:30:00  

In [44]:
# Cell 7: Do financial calculations
# Scale totals to 30 years (multiply by 10 as per your approach)
print("Testing financial calculation")

try:
    # Scale the 3-year totals to 30 years
    totals_30yr = {k: v * 10 for k, v in totals.items()}
    
    # Calculate financials
    fin_results = compute_financials(
        totals=totals_30yr,
        battery_kwh=test_battery_kwh,
        pv_kw=total_pv_capacity,
        discount_rate=0.05  # 5% discount rate
    )
    
    # Display results
    print(f"Financial calculation complete")
    print(f"  - CAPEX (PV): ${fin_results['capex_pv']:,.2f}")
    print(f"  - CAPEX (Battery): ${fin_results['capex_battery']:,.2f}")
    print(f"  - Total CAPEX: ${fin_results['capex_total']:,.2f}")
    print(f"  - Export revenue: ${fin_results['export_revenue_total']:,.2f}")
    print(f"  - Import cost: ${fin_results['import_cost_total']:,.2f}")
    print(f"  - OPEX: ${fin_results['opex_total']:,.2f}")
    print(f"  - Net cost: ${fin_results['net_cost']:,.2f}")
    print(f"  - IRR: {fin_results['irr']*100:.2f}%")
    if fin_results['npv'] is not None:
        print(f"  - NPV: ${fin_results['npv']:,.2f}")
        print("PETER: CHECK AGAIN IRR if NEGATIVE, wont work, remember to redo - and change NPV to NPC")
    
except Exception as e:
    print(f"Error in financial calculation - check again (or redo calcs): {e}")
   

Testing financial calculation
Financial calculation complete
  - CAPEX (PV): $10,000.00
  - CAPEX (Battery): $1,500.00
  - Total CAPEX: $11,500.00
  - Export revenue: $366.47
  - Import cost: $196,513.26
  - OPEX: $10,950.00
  - Net cost: $218,596.79
  - IRR: nan%
  - NPV: $-117,619.51
PETER: CHECK AGAIN IRR if NEGATIVE, wont work, remember to redo - and change NPV to NPC


In [45]:
# Cell 8: Test the objective function
print("Testing objective function...")

try:
    # Create test parameters
    test_params = {
        'battery_kwh': test_battery_kwh,
        'pv_kw': total_pv_capacity
    }
    
    # Evaluate the objective function
    objective_results = evaluate_solution(
        params=test_params,
        pv_profile=pv_profile,
        demand_profile=demand_profile
    )
    
    # Display results
    print(f"Objective function evaluation successful")
    print(f"  - Negative IRR: {objective_results[0]}")
    print(f"  - NPC: ${objective_results[1]:,.2f}")
    print(f"  - Actual IRR: {-objective_results[0]*100:.2f}%")
    
except Exception as e:
    print(f"Error in objective function: {e}")
    raise

Testing objective function...
Objective function evaluation successful
  - Negative IRR: nan
  - NPC: $218,596.79
  - Actual IRR: nan%


In [49]:
# Cell 9: Define the optimisation problem to solve
print("PETER: ADD SOLAR PV TO VARY FOR EACH ROOF")
print("PETER: TO ADD EACH SIMULATION TO SAY TIME TAKEN")
print("Defining optimization problem")

try:
    # Define the NSGA-II optimization problem (Remember peter to write optimisation with s as python uses american english)
    class BatteryOptimizationProblem(Problem):
        def __init__(self):
            # Define one variable (battery size in kWh)
            # with lower bound 0 and upper bound 50
            super().__init__(
                n_var=1,             # Number of decision variables
                n_obj=2,             # Number of objectives
                n_ieq_constr=0,      # Number of inequality constraints
                xl=np.array([0.0]),  # Lower bounds of variables
                xu=np.array([50.0])  # Upper bounds of variables
            )
            # Store the profiles for use in evaluation
            self.pv_profile = pv_profile
            self.demand_profile = demand_profile
        
        def _evaluate(self, x, out, *args, **kwargs):
            # Evaluate each solution in the population
            n_solutions = x.shape[0]
            F = np.zeros((n_solutions, 2))  # For storing [-IRR, NPC] values
            
            # Loop through each solution (battery size)
            for i in range(n_solutions):
                battery_kwh = x[i, 0]
                # Define parameters for evaluation
                params = {
                    'battery_kwh': battery_kwh,
                    'pv_kw': total_pv_capacity
                }
                # Evaluate the solution using your objective function
                # returns [-IRR, NPC]
                F[i, :] = evaluate_solution(params, self.pv_profile, self.demand_profile)
            
            # Set the output objectives array
            out["F"] = F
    
    # Create an instance of the problem
    problem = BatteryOptimizationProblem()
    print("Optimisation problem defined")
    
except Exception as e:
    print(f"Error defining optimization problem: {e}")
    raise

PETER: ADD SOLAR PV TO VARY FOR EACH ROOF
PETER: TO ADD EACH SIMULATION TO SAY TIME TAKEN
Defining optimization problem
Optimisation problem defined


In [ ]:
# Cell 10: Run NSGA‑II optimisation and save Pareto front
print("Running NSGA‑II optimisation…")

# Create problem & algorithm
problem   = BatteryOptimizationProblem()
algorithm = NSGA2(pop_size=40)
termination = get_termination("n_gen", 50)

# Run
res = minimize(
    problem,
    algorithm,
    termination,
    seed=42,
    verbose=True
)

# Extract Pareto‐optimal solutions
battery_sizes = res.X.flatten()
pareto_F      = res.F           # shape (n_solutions,2): [-IRR, NPC]
irr_vals      = -pareto_F[:, 0] # back to +IRR
npc_vals      = pareto_F[:, 1]

# Build DataFrame & save
import pandas as pd
df = pd.DataFrame({
    'battery_kwh': battery_sizes,
    'IRR':          irr_vals,
    'NPC':          npc_vals
})

out_csv = out_dir / 'pareto_solutions.csv'
df.to_csv(out_csv, index=False)
print(f"✅ Pareto front saved to {out_csv}")


In [50]:
# Cell 11: Visualise/showcase result
# PeTER fix: the output not defiend, re do but I want it in a seperate, also we want the half hourly results for 30 years to proof with import and export, 
# would be beneficial to have, in csv
print("Creating visualization of Pareto front...")

try:
    import matplotlib.pyplot as plt
    
    # Plot the Pareto front
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(df['IRR'] * 100, df['NPC'] / 1000, 
                          c=df['battery_kwh'], cmap='viridis', 
                          s=100, alpha=0.7)
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Battery Size (kWh)')
    
    # Add labels for points
    for i, row in df.iterrows():
        plt.annotate(f"{row['battery_kwh']:.1f} kWh", 
                     (row['IRR'] * 100, row['NPC'] / 1000),
                     xytext=(5, 5), textcoords='offset points')
    
    # Add labels and title
    plt.xlabel('IRR (%)')
    plt.ylabel('Net Present Cost ($ thousands)')
    plt.title('Pareto Front: IRR vs. NPC for Different Battery Sizes')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Save the figure
    plt.tight_layout()
    out_fig = out_dir / 'pareto_front.png'
    plt.savefig(out_fig, dpi=300)
    plt.show()
    
    print(f"Visualization saved to {out_fig}")
    
except Exception as e:
    print(f"Error creating visualization: {e}")
    
print("\nAnalysis complete!")

Creating visualization of Pareto front...
Error creating visualization: 'WindowsPath' object is not subscriptable

Analysis complete!


<Figure size 1000x600 with 0 Axes>

In [ ]:
# ─── Cell 12: Export all inputs & results to CSV ────────────────────────────────
import pandas as pd

# 1) Roof configuration
pd.DataFrame(roof_params) \
  .to_csv(out_dir / 'roof_config.csv', index=False)

# 2) Weather files list
pd.DataFrame({'weather_file': weather_files}) \
  .to_csv(out_dir / 'weather_files.csv', index=False)

# 3) Demand file used
pd.DataFrame({'demand_file': [str(demand_file)]}) \
  .to_csv(out_dir / 'demand_file.csv', index=False)

# 4) Simulation parameters
pd.DataFrame({
    'repeats_per_file': [REPEATS_PER_FILE],
    'start_years': [start_years]
}) \
  .to_csv(out_dir / 'simulation_params.csv', index=False)

# 5) Dispatch totals (3‑year, pre‑scaling)
pd.DataFrame.from_dict(totals, orient='index', columns=['value']) \
  .to_csv(out_dir / 'dispatch_totals.csv')

# 6) Financial results
pd.DataFrame.from_dict(fin_results, orient='index', columns=['value']) \
  .to_csv(out_dir / 'financial_summary.csv')

# 7) Pareto front
df.to_csv(out_dir / 'pareto_front.csv', index=False)

print(f"All summary CSVs written to {out_dir}")
